# Assignment 5: Molecular Dynamics

---

Molecular dynamics (MD) is a method for simulating the movements of atoms and molecules and is wildly used to study properties of molecular systems in chemical physics, materials science, and biophysics. Hereby, the system is propagated by solving its equations of motion. In the most common cases, the trajectories follow simply the numerical solutions of Newton's equation of motion, where usually the Leapfrog or Velocity-Verlet scheme are used to propagte the system.


In the following you are asked to write a small MD code, that uses the Velocity-Verlet alogrithm to propagate a molecular system:

\begin{align}
x(t) & = x_0 + v(t)dt + 0.5 a(t)dt^2 \\
a(t) & = F(x(t))/m \\
v(t) & = v_0 + 0.5(a(t) + a_0)dt \\
\end{align}

The code should read in the initial structure and optional the velocities from an 'XYZ'-format file, example for $H_2$ below, then propagate the system. For the computation of the forces on the atoms you should use a common morse-potential.

The code should print three energies: total energy (for check on energy conservation), kinetic energy, potential energy of the system as well as optionally the trajectory of structures in XYZ-format.

In [50]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
# Get our data from github
if not os.path.exists("../../python_lessons/"):
  #Move to the folder of assignment 5
  !git clone https://github.com/MFSJMenger/python_lessons.git
  os.chdir("python_lessons/assignment5/")

In [51]:
#defining the constants used

m = 1.0079
De = 0.176
a = 1.02
Re = 1.40
v0 = 0
x0 = 0

In [52]:
# Reading the xyzfile and extracting the information needed

def read_xyz(xyzfile):                # Define a function that takes the arguement 'xyzfile' which is the file name of an XYZ file.
  with open(xyzfile, 'r') as fh:      # opens the XYZ file in read mode and further names it as fh. The'with' statement ensures that the file is properly closed after its suite finishes
    xyz = fh.readlines()              # reads the lines from fh and stores them as a list of strings in 'xyz'

  # Read the number of atoms
  nAtoms = int(xyz[0])                 # extracts the first line of the file and converts it into an integer. The first line contains the number of atoms in the molecule
  masses = np.zeros(nAtoms)            # creates an array named 'masses' which is initialized to zero for storing the masses of each atom.
  geom = np.zeros((nAtoms,3))          # creates an array with dimensions ( , ), initialized to zero for storing the atomic coordinates
  i = 0                                # a counter to keep track of the atom being processed.

  for line in xyz[2:]:                                # iterates over each line in the 'xyz' list starting from the third line
   line_split = line.split()                          # splits each line into a list of strings based on whitespace. It seperates the atom symbol and its coordinates.
   if len(line_split)>=4:
    masses[i] = m                                     # assigns value m to the ith element of masses array
    geom[i] = [float(x) for x in line_split[1:]]      # converts the coordinates to floats and assigns them to the 'geom' array at index 'i'.
    i+=1                                              # increments the counter 'i' to move to the next atom.
  return nAtoms, masses, geom                         # returns the number of atoms, the array of masses and the array of atomic coordinates

print(read_xyz('h2.xyz'))                             # demonstration

(2, array([1.0079, 1.0079]), array([[0. , 0. , 0. ],
       [0.9, 0. , 0. ]]))


In [53]:
# Defining the morse potential and the gradient of the potential

def morse_potential(R,De,a,Re):
  V = De*(1-np.exp(-a*(R-Re)))**2
  F = -2*a*De*(1-np.exp(-a*(R-Re)))*np.exp (-a*(R-Re))
  return V,F

In [54]:
# Defining the velocity verlet algorithm

def velocity_verlet(x,v,masses,t,De,a,Re,):

  # initialization
  nAtoms = len(x)                               # determining the number of atoms
  forces = np.zeros((nAtoms,3))                 # an array to store forces on each atoms
  VE = 0.0                                      # initial potential energy = 0

  # Calculation of force and potential energy
  for i in range(nAtoms):                       # iterates over all the atoms
    for j in range(i+1,nAtoms):
      R = np.linalg.norm(x[i]-x[j])             # calculates the distance between atoms
      V,F = morse_potential(R,De,a,Re)          # calcuates the potential and gradient using the Morse Potential
      force = F * (x[i]-x[j])/R                 # calculates the force
      forces[i] +=force                         # adds the force calculated on atom i to the array
      forces[j] -= force                        # substracts the force calculated on atom j from the array because the forces acting on i and j are in opposite direction
      VE += V                                   # adding the potential energy

  # Reshaping the mass array to match that of forces calculated
  m_re = masses[:,np.newaxis]

  # Updating the positions according to the Velocity-verlet algorithm
  x += x0 + v*t + 0.5*forces/m_re*t**2

  # Calculating the new forces after updating the positions
  n_forces = np.zeros((nAtoms,3))
  for i in range(nAtoms):
    for j in range(i+1,nAtoms):
      R = np.linalg.norm(x[i]-x[j])
      V,F = morse_potential(R,De,a,Re)
      force = F * (x[i]-x[j])/R
      n_forces[i] +=force
      n_forces[j] -= force

  # updating the velocity according to the Velocity-verlet algorithm
  v += v0 + (forces/m_re + n_forces/m_re)*t

  return x,v,n_forces,VE

In [55]:
# The molecular dynamics simulation

# defining the MD functional
def MD(xyzfile,num,t):
  nAtoms, masses,x = read_xyz(xyzfile)                         # reading the xyz file
  v = np.zeros_like(x)                                         # initializing velocity

  # Iteration of the simulation over a specific number of steps
  for step in range(num):
    x,v,forces,VE = velocity_verlet(x,v,masses,t,De,a,Re)

    m_re = masses[:,np.newaxis]                               # reshaping the mass array to calculate the kinetic energy

    # Calculating the energies
    KE = 0.5*np.sum(m_re*v**2)
    T = KE + VE

    print(f"Step{step+1}:Kinetic energy = {KE},Potential energy = {VE},Total energy = {T}")

In [56]:
# Running the simulation

xyzfile = 'h2.xyz'
num = 4000
t = 0.01
MD(xyzfile,num,t)

Step1:Kinetic energy = 6.278716820003316e-05,Potential energy = 0.07789977783676441,Total energy = 0.07796256500496444
Step2:Kinetic energy = 0.00025104246435781326,Potential energy = 0.07788407993792892,Total energy = 0.07813512240228673
Step3:Kinetic energy = 0.0005643945456720524,Potential energy = 0.07780563027991592,Total energy = 0.07837002482558798
Step4:Kinetic energy = 0.0010021906329447808,Potential energy = 0.07766459690410624,Total energy = 0.07866678753705102
Step5:Kinetic energy = 0.001563499035236845,Potential energy = 0.07746128869534391,Total energy = 0.07902478773058076
Step6:Kinetic energy = 0.002247112783802489,Potential energy = 0.07719615417905293,Total energy = 0.07944326696285542
Step7:Kinetic energy = 0.003051554351952518,Potential energy = 0.07686977976258222,Total energy = 0.07992133411453474
Step8:Kinetic energy = 0.0039750814305183355,Potential energy = 0.07648288743210359,Total energy = 0.08045796886262192
Step9:Kinetic energy = 0.005015693721904103,Potent